In [ ]:
!pip install tiktoken
!pip install PyMuPDF
!pip install fpdf
!pip install mistralai
!pip install llama-index-multi-modal-llms-mistralai
!pip install matplotlib
!pip install yfinance
!pip install beautifulsoup4
!pip install fpdf

In [ ]:
import requests
import fitz
import io
import yfinance as yf
from fpdf import FPDF
import matplotlib.pyplot as plt
import pandas as pd
import os


In [ ]:
# API Key Setup
from google.colab import userdata
api_key = "DNMDNEITWbr4kUjWN3pDv23kjYAHr8vT"
os.environ["MISTRAL_API_KEY"] = api_key
from mistralai import Mistral

model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [ ]:
from fpdf import FPDF
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import requests
import io
import fitz
from mistralai import Mistral  # Updated import

def get_pdf_text(url):
    """
    Extracts text from a PDF file accessed via a URL.
    """
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    with fitz.open(stream=filestream, filetype="pdf") as doc:
        detail_judgement = ""
        for page in doc:
            detail_judgement += page.get_text()
    return detail_judgement

def summarize_report(text):
    """
    Summarizes a given text using a Mistral model.
    """
    prompt = f"""
    You are a financial assistant. Analyze and summarize the following document content with key insights and suggestions for better understanding:
    {text}
    """



    # Generate a response using Mistral (API usage is hypothetical; adjust accordingly)
    chat_response = client.chat.complete(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
    )


    # Assuming Mistral returns a structure similar to OpenAI:
    # response["choices"] might be a list of generated answers
    return chat_response.choices[0].message.content

def sp500_return():
    """
    Calculates the approximate 3-year historical return of the S&P 500 Index.
    """
    sp500 = yf.Ticker("^GSPC")
    hist = sp500.history(period="5y")

    if hist.empty:
        raise ValueError("No historical data retrieved for the S&P 500.")

    three_years_ago = hist.index[-1] - pd.DateOffset(years=3)
    recent_hist = hist[hist.index >= three_years_ago]

    if recent_hist.empty:
        raise ValueError("No data available for the specified 3-year period.")

    start_price = recent_hist["Close"].iloc[0]
    end_price = recent_hist["Close"].iloc[-1]

    return_rate = ((end_price - start_price) / start_price) * 100
    risk = "High"
    return return_rate, risk

def bond_return():
    """
    Provides a hypothetical 3-year return for government bonds.
    """
    annual_return = 3
    total_return = ((1 + (annual_return / 100)) ** 3 - 1) * 100
    risk = "Low"
    return total_return, risk

def high_yield_savings_return():
    """
    Provides a hypothetical 3-year return for a high-yield savings account.
    """
    annual_return = 2
    total_return = ((1 + (annual_return / 100)) ** 3 - 1) * 100
    risk = "Very Low"
    return total_return, risk

def savings_return():
    """
    Provides a hypothetical 3-year return for a standard savings account.
    """
    annual_return = 0.5
    total_return = ((1 + (annual_return / 100)) ** 3 - 1) * 100
    risk = "Very Low"
    return total_return, risk

def clean_text(text):
    return text.encode('latin1', 'replace').decode('latin1')

def generate_pdf(name, starting_amount, sp500, sp500_risk, bond, bond_risk, high_yield, high_yield_risk, savings, savings_risk, summary):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Header
    pdf.cell(200, 10, txt=clean_text(f"Investment Analysis Report for {name}"), ln=True, align="C")
    pdf.ln(10)

    # Details
    pdf.cell(200, 10, txt=clean_text(f"Initial Investment: ${starting_amount:,.2f}"), ln=True)
    pdf.cell(200, 10, txt="Investment Period: 3 Years", ln=True)
    pdf.ln(10)

  # gloab out look
    pdf.ln(10)
    pdf.cell(200, 10, txt="Curent Global Analyzed:", ln=True)
    pdf.set_font("Arial", size=11)
    pdf.multi_cell(0, 10, clean_text(summary))
    pdf.ln(10)


    # Investment options
    pdf.cell(200, 10, txt="US Investment Options:", ln=True)

    sp500_end = starting_amount * (1 + sp500 / 100)
    pdf.cell(200, 10, txt=clean_text(f"1. S&P 500 Index Fund: {sp500:.2f}% Return (Risk: {sp500_risk})"), ln=True)
    pdf.cell(200, 10, txt=clean_text(f"   End Amount: ${sp500_end:,.2f}"), ln=True)

    bond_end = starting_amount * (1 + bond / 100)
    pdf.cell(200, 10, txt=clean_text(f"2. Government Bonds: {bond:.2f}% Return (Risk: {bond_risk})"), ln=True)
    pdf.cell(200, 10, txt=clean_text(f"   End Amount: ${bond_end:,.2f}"), ln=True)

    high_yield_end = starting_amount * (1 + high_yield / 100)
    pdf.cell(200, 10, txt=clean_text(f"3. High-Yield Savings: {high_yield:.2f}% Return (Risk: {high_yield_risk})"), ln=True)
    pdf.cell(200, 10, txt=clean_text(f"   End Amount: ${high_yield_end:,.2f}"), ln=True)

    savings_end = starting_amount * (1 + savings / 100)
    pdf.cell(200, 10, txt=clean_text(f"4. Regular Savings: {savings:.2f}% Return (Risk: {savings_risk})"), ln=True)
    pdf.cell(200, 10, txt=clean_text(f"   End Amount: ${savings_end:,.2f}"), ln=True)


    # Generate a bar chart of returns
    plt.figure()
    options = ["S&P 500", "Bonds", "High-Yield Savings", "Regular Savings"]
    returns = [sp500, bond, high_yield, savings]
    plt.bar(options, returns)
    plt.xlabel("Investment Options")
    plt.ylabel("Returns (%)")
    plt.title("3-Year Investment Returns")
    chart_path = "investment_chart.png"
    plt.savefig(chart_path)
    plt.close()

    # Add chart to PDF
    pdf.image(chart_path, x=10, y=pdf.get_y() + 10, w=190)
    pdf.output("Investment_Report.pdf")
    print("Report generated: Investment_Report.pdf")

def start(name, amount, pdf_url):
    """
    Main execution function:
    - Fetches the text from the given PDF URL.
    - Summarizes the PDF text using the Mistral model.
    - Fetches returns for various investment options.
    - Generates the PDF report with the summary included.
    """
    # Get and summarize the PDF content
    text = get_pdf_text(pdf_url)
    summary = summarize_report(text)

    # Get investment returns and risks
    sp500, sp500_risk = sp500_return()
    bond, bond_risk = bond_return()
    high_yield, high_yield_risk = high_yield_savings_return()
    savings, savings_risk = savings_return()

    # Generate final PDF with summary
    generate_pdf(name, amount, sp500, sp500_risk, bond, bond_risk, high_yield, high_yield_risk, savings, savings_risk, summary)

if __name__ == "__main__":
    # Example usage:
    pdf_url = " https://raw.githubusercontent.com/gheniabla/datasets/refs/heads/master/economicoutlook.pdf"
    start("John Doe", 200000, pdf_url)


Report generated: Investment_Report.pdf
